In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import scipy
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image
from sklearn.model_selection import train_test_split
from scipy import stats

%matplotlib inline
matplotlib.style.use('bmh')

In [58]:
data_all_thomas = pd.read_excel('annotations/data_all_thomas.xlsx')
data_labels_video_features = pd.read_csv('annotations/combined_labels_video_features.csv')
labels = data_labels_video_features[['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val','gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val']]
labels_max = labels[['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val']]
labels_min = labels[['gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val']]
video_features = data_labels_video_features.drop(['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val','gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val'],axis=1)

#Applying z normalization
video_features.apply(stats.zscore)

#Applying L2 normalization
video_features = pd.DataFrame(sk.preprocessing.normalize(video_features, norm='l2',axis=1))


In [ ]:
#sum of feature vector should be 1 after l2 normalization
sum_of = sum(map(lambda x : x*x,video_features.iloc[3]))

In [ ]:
video_features.head()

In [ ]:
#Random forest
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt')
rf.fit(video_features,labels_max)

In [ ]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier


tree = DecisionTreeClassifier(random_state=50)
tree.fit(video_features, labels_max)
print(f'Decision tree has {tree.tree_.node_count} nodes with maximum depth {tree.tree_.max_depth}.')
export_graphviz(tree, 'tree.dot', rounded = True, feature_names = features, class_names = ['0', '1','2'], filled = True)
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=400']);

In [ ]:
#Support Vector Machine
from sklearn.metrics import classification_report, confusion_matrix,recall_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


X_train, X_test, y_train, y_test = train_test_split(video_features, labels['gold_gt_min_aro'], train_size=660,test_size=300,shuffle=False)

#Gridsearching best parameters
param_grid = [{ 'kernel': ['linear'],'tol':[1,0.1,0.01,0.001,0.0001]}]
#param_grid = [{'C': [1,0.00001], 'kernel': ['linear']}]
svc = SVC()
#Scoring based on recall macro average score
clf = GridSearchCV(svc,param_grid, scoring='recall_macro')
clf.fit(X_train,y_train)
results = clf.cv_results_

In [ ]:
#best estimator based on param grid search
y_pred = clf.best_estimator_.predict(X_test)
report_best_estimator = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()


In [65]:
#multiple SVM
from sklearn.metrics import classification_report, confusion_matrix,recall_score
from sklearn.svm import SVC


variables = [labels['gold_gt_min_aro'],labels['gold_gt_min_like'],labels['gold_gt_min_val'],data_all_thomas['arousal'],data_all_thomas['likeability'],data_all_thomas['valence']]
hyperparams_c = [100000,10000,1000,100,10,1,0.1,0.01,0.001,0.0001,0.00001]
gammas = [100000,10000,1000,100,10,1,0.1,0.01,0.001,0.0001,0.00001]
report_svms = pd.DataFrame(columns=['variable_name'] + gammas)

for variable in variables:
    row = [variable.name]
    for gamma_value in gammas:
        X_train, X_test, y_train, y_test = train_test_split(video_features, variable, train_size=660,test_size=300,shuffle=False)
        clf_single = SVC(kernel='rbf', C=10, gamma=gamma_value)
        clf_single.fit(X_train,y_train)
        y_pred_single = clf_single.predict(X_test)
        row = row + [recall_score(y_test, y_pred_single,average='macro')]
        #report_svms = report_svms.append([[variable.name]])
        #report_svms = report_svms.append([recall_score(y_test, y_pred_single,average='macro')])
        #report_svms = report_svms.append(pd.DataFrame(classification_report(y_test,y_pred_single, output_dict=True)).transpose())
    report_svms = report_svms.append(pd.Series(row,index=report_svms.columns),ignore_index=True)

report_svms.to_csv('report_svms_c_l2_zn_rbf.csv')



In [68]:
#single SVM
from sklearn.metrics import classification_report, confusion_matrix,recall_score
from sklearn.svm import SVC


X_train, X_test, y_train, y_test = train_test_split(video_features, data_all_thomas['interview'] , train_size=660,test_size=300,shuffle=False)
clf_single = SVC(kernel='linear', C=1)
clf_single.fit(X_train,y_train)
y_pred_single = clf_single.predict(X_test)
recall_score(y_test, y_pred_single,average='macro')

ValueError: Unknown label type: 'continuous'

In [ ]:
#Logistic regression
from sklearn.linear_model import LogisticRegression


X_train, X_test, y_train, y_test = train_test_split(video_features, labels['gold_gt_min_val'], train_size=660,test_size=300,shuffle=False)
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
report = classification_report(y_test,y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv('report.csv')

In [ ]:
#Boxplots
def save_boxplot(x,y):
    box_data_1 = data_all_thomas[data_all_thomas[x] == 1 ].get(y)
    box_data_2 = data_all_thomas[data_all_thomas[x] == 2 ].get(y)
    box_data_3 = data_all_thomas[data_all_thomas[x] == 3 ].get(y)
    box_data = [box_data_1,box_data_2,box_data_3]
    fig, ax = plt.subplots()
    ax.boxplot(box_data)
    ax.set_ylabel(y)
    ax.set_xlabel(x)
    fig.savefig('boxplots/boxplot_'+x+'_'+y+'.svg')


def create_boxplots():
    classifications = ['agreeableness','conscientiousness','extraversion','interview','neuroticism','openness']
    variables = ['valence','arousal','likeability']
    for x in variables:
        for y in classifications:
            save_boxplot(x,y)

In [ ]:
#Basis statistics
def stats(x):
    data_class_1 = data_all_thomas[data_all_thomas[x] == 1 ].get('interview')
    data_class_2 = data_all_thomas[data_all_thomas[x] == 2 ].get('interview')
    data_class_3 = data_all_thomas[data_all_thomas[x] == 3 ].get('interview')
    print('data1: mean=%.3f stdv=%.3f' % (np.mean(data_class_1), np.std(data_class_1)))
    print('data2: mean=%.3f stdv=%.3f' % (np.mean(data_class_2), np.std(data_class_2)))
    print('data3: mean=%.3f stdv=%.3f' % (np.mean(data_class_3), np.std(data_class_3)))    
    stat, p = ttest_ind(data_class_2, data_class_3) 
    print(stat, p)

In [ ]:
#Statistical significance
def mannWithNeyuScore():
    data_class_1 = data_all_thomas[data_all_thomas['likeability'] == 1 ].get('interview')
    data_class_2 = data_all_thomas[data_all_thomas['likeability'] == 2 ].get('interview')
    data_class_3 = data_all_thomas[data_all_thomas['likeability'] == 3 ].get('interview')
    scipy.stats.mannwhitneyu(data_class_1, data_class_3)